In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import cv2 
import scipy

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

import os
import os.path
import random 

from math import sqrt
from skimage import data
from skimage.feature import blob_log
from skimage.color import rgb2gray

In [ ]:
%matplotlib inline 

# Image Processing

In [ ]:
#The following code reads the image and performs laplacian of gaussian computer vision algorithm. 
#The graph is also created as a part of the function. 
from PIL import Image

def check_shape(image, num):
    image = cv2.imread(image)
    height, width, channels = image.shape 

    if (height*width > 330000000):
        return large_classify(image, num)
    else:
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    
        contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
    
        image = image[y:y+h,x:x+w]
        return small_classify(image, num)

def splice_image(image, x, y):
    test = image
    my_slice = test[x:y, x:y]
    return my_slice

#Pick random number of positions in image and detects blobs 
def large_classify(image, num):
    array = []
    #image = cv2.imread(image)
    height, width, channels = image.shape
    if(width>15000):
        image = cv2.resize(image, dsize=(10000, 10000), interpolation=cv2.INTER_LINEAR)
    placer = num - 1
    X = width/300
    Xfirst = 0
    Y = X+height/300
    Yfirst = X
    for i in range(placer):
        x_pick = int(random.uniform(Xfirst, X))
        y_pick = int(random.uniform(Yfirst, Y))
        array.append(LOG(splice_image(image, x_pick, y_pick)))
        Xfirst = X
        Yfirst = Y
        X = X + width/300
        Y = X+height/300
    return array

def small_classify(image, num):
    array = []
    #image = cv2.imread(image)
    height, width, channels = image.shape
    placer = num - 1
    X = width/50
    Xfirst = 0
    Y = X+height/50
    Yfirst = X
    for i in range(placer):
        x_pick = int(random.uniform(Xfirst, X))
        y_pick = int(random.uniform(Yfirst, Y))
        array.append(LOG(splice_image(image, x_pick, y_pick)))
        Xfirst = X
        Yfirst = Y
        X = X + width/50
        Y = X+height/50
    return array

In [ ]:
import matplotlib.pyplot as plt

def LOG(image): 
    #Laplacian: Highlights regions of rapid intensity change
    blobsLOG = blob_log(rgb2gray(image), max_sigma=30, num_sigma=10, threshold=.1)

    # Computing radii
    blobsLOG[:, 2] = blobsLOG[:, 2] * sqrt(2)


    #Setting plot
    blobList = [blobsLOG]
    colors = ['yellow']
    titles = ['Laplacian of Gaussian']
    sequence = zip(blobList, colors, titles)
    countofCircles = 0
    
    for index, (blobs, color, title) in enumerate(sequence):
        for blob in blobs:
            countofCircles=countofCircles+1
        
    return countofCircles 

def decide(image):
    count = LOGandDOG(image) 
    if (count > 3000):
        return('The image is focused.')
    else:
        return('The image is not focused.')


In [ ]:
#Loading Data 

In [ ]:
#Reading in Images
train_focus_path = "Testing_Focus_Images/"
train_outfocus_path = "Testing_Outfocus_Images/"

training_set = []
labels = []

for filename in os.listdir(train_outfocus_path):
    print(filename)
    data_set = []
    imagepath = train_outfocus_path + filename
    data_set = find_blobs_out(imagepath, 7)
    training_set.append(data_set)
    labels.append(0)

for filename in os.listdir(train_focus_path):
    print(filename)
    data_set = []
    imagepath = train_focus_path + filename
    data_set = find_blobs_focus(imagepath, 7)
    training_set.append(data_set)
    labels.append(1)
    

In [ ]:
#Neural Network to Train Based on Blob Samples
training_set

In [ ]:
#Small Neural Network
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(training_set)
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

In [ ]:
from keras import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_dim=6, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))


In [ ]:
#Compiling the neural network
model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [ ]:
#Fitting the data to the training dataset
model.fit(X_train,y_train, batch_size=10, epochs=10)

In [ ]:
#print model accuracy
print(accuracy_score(test_labels, preds))

In [ ]:
def LOGandDOG_Predict(image):
    np = []
    data_set = check_shape(image)
    np.append(data_set)
    preds = gnb.predict(np)
    if (preds==1):
        return True
    else: 
        return False
    
def LOGandDOG_Predict(image, num):
    np = []
    data_set = check_shape(image, num)
    np.append(data_set)
    preds = 0
    print(np)
    for i in data_set:
        if i > 1000:
            preds = preds+1
    if (preds > 2):
        return True
    else: 
        return False

In [ ]:
def analyze_image(image, num):
    return (LOGandDOG_Predict(image, num))


In [ ]:
def imagesize(image):
    image = cv2.imread(image)
    height, width, channels = image.shape 
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    
    image = image[y:y+h,x:x+w]
    plt.imshow(image)
    plt.show()
    
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    
    image = image[y:y+h,x:x+w]

In [ ]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model.h5')